In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
veriler = pd.read_csv("Churn_Modelling.csv")

In [3]:
veriler.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
X = veriler.iloc[:,3:13].values
Y = veriler.iloc[:,13].values

In [7]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
X[:,1] = le.fit_transform(X[:,1])
X[:,2] = le.fit_transform(X[:,2])

In [8]:
X

array([[619, 0, 0, ..., 1, 1, 101348.88],
       [608, 2, 0, ..., 0, 1, 112542.58],
       [502, 0, 0, ..., 1, 0, 113931.57],
       ...,
       [709, 0, 0, ..., 0, 1, 42085.58],
       [772, 1, 1, ..., 1, 0, 92888.52],
       [792, 0, 0, ..., 1, 0, 38190.78]], dtype=object)

ülkeler arasında anlamlı bir ilişki yok 1 0'dan büyüktür gibi bu yüzden bu değerleri aynı zamanda encode ediyoruz

In [10]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

ohe = ColumnTransformer([("ohe", OneHotEncoder(dtype=float),[1])],remainder="passthrough")
X=ohe.fit_transform(X)
X=X[:,1:]

In [11]:
X

array([[0.0, 0.0, 619, ..., 1, 1, 101348.88],
       [0.0, 1.0, 608, ..., 0, 1, 112542.58],
       [0.0, 0.0, 502, ..., 1, 0, 113931.57],
       ...,
       [0.0, 0.0, 709, ..., 0, 1, 42085.58],
       [1.0, 0.0, 772, ..., 1, 0, 92888.52],
       [0.0, 0.0, 792, ..., 1, 0, 38190.78]], dtype=object)

In [12]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size=0.33, random_state=0)

In [13]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

X_train = sc.fit_transform(x_train)
X_test = sc.transform(x_test)

In [16]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [24]:
classifier = Sequential() # yapay sinir ağın oluştu ama içi boş
classifier.add(Dense(6, activation="relu", input_dim = 11)) # giriş katmanı

units -> gizli katman sayısı. bu sayının kaç olacağı sana bağlı 
ama genellikle giriş katmanındaki nöron sayısı/2 oluyo." = 6

init -> nn yapısını initilaze etmek. yapay sinir ağının ilk değerlerin atanması.
(bu değerlerin 0'a yakın olması gerekir). / init="uniform" eski sürüm gerek yok kendisi yazıyo

input dim -> giriş katmanı kaç nöronlu olacak

In [25]:
classifier.add(Dense(6, activation="relu")) # gizli katman

In [27]:
classifier.add(Dense(1,activation="sigmoid")) # çıkış katmanı

In [30]:
classifier.compile(loss="binary_crossentropy", metrics=["accuracy"])
# optimizer="adam" -> eski sürüm

adam, stochastic gradient descent yöntemi bir versiyonu

loss -> sistem çalışırken nasıl bir fonksiyon kullansın. bizim çıktı binary 0-1 arasında olduğu için binary_crossentropy tercih edildi.

In [32]:
classifier.fit(X_train,y_train, epochs=50)

Epoch 1/50
210/210 [==============================] - 0s 2ms/step - loss: 0.4910 - accuracy: 0.7979
Epoch 2/50
210/210 [==============================] - 0s 2ms/step - loss: 0.4595 - accuracy: 0.7979
Epoch 3/50
210/210 [==============================] - 0s 2ms/step - loss: 0.4438 - accuracy: 0.7979
Epoch 4/50
210/210 [==============================] - 0s 2ms/step - loss: 0.4352 - accuracy: 0.7979
Epoch 5/50
210/210 [==============================] - 0s 2ms/step - loss: 0.4297 - accuracy: 0.7979
Epoch 6/50
210/210 [==============================] - 0s 2ms/step - loss: 0.4252 - accuracy: 0.7979
Epoch 7/50
210/210 [==============================] - 0s 2ms/step - loss: 0.4208 - accuracy: 0.7979
Epoch 8/50
210/210 [==============================] - 0s 2ms/step - loss: 0.4164 - accuracy: 0.8055
Epoch 9/50
210/210 [==============================] - 0s 2ms/step - loss: 0.4114 - accuracy: 0.8213
Epoch 10/50
210/210 [==============================] - 0s 2ms/step - loss: 0.4063 - accuracy: 0.8234

In [33]:
y_pred = classifier.predict(X_test)

104/104 [==============================] - 0s 2ms/step


In [34]:
y_pred = (y_pred > 0.5)

In [36]:
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [ True],
       [False]])

In [37]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)
print(cm)

[[2505  112]
 [ 359  324]]
